# Advanced Retrieval with LangChain

In the following notebook, we'll explore various methods of advanced retrieval using LangChain!

We'll touch on:

- Naive Retrieval
- Best-Matching 25 (BM25)
- Multi-Query Retrieval
- Parent-Document Retrieval
- Contextual Compression (a.k.a. Rerank)
- Ensemble Retrieval
- Semantic chunking

We'll also discuss how these methods impact performance on our set of documents with a simple RAG chain.

There will be two breakout rooms:

- 🤝 Breakout Room Part #1
  - Task 1: Getting Dependencies!
  - Task 2: Data Collection and Preparation
  - Task 3: Setting Up QDrant!
  - Task 4-10: Retrieval Strategies
- 🤝 Breakout Room Part #2
  - Activity: Evaluate with Ragas

# 🤝 Breakout Room Part #1

## Task 1: Getting Dependencies!

We're going to need a few specific LangChain community packages, like OpenAI (for our [LLM](https://platform.openai.com/docs/models) and [Embedding Model](https://platform.openai.com/docs/guides/embeddings)) and Cohere (for our [Reranker](https://cohere.com/rerank)).

We'll also provide our OpenAI key, as well as our Cohere API key.

In [3]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass.getpass("Enter your OpenAI API Key:")

In [4]:
os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY") or getpass.getpass("Cohere API Key:")

## Task 2: Data Collection and Preparation

We'll be using our Use Case Data once again - this time the strutured data available through the CSV!

### Data Preparation

We want to make sure all our documents have the relevant metadata for the various retrieval strategies we're going to be applying today.

In [5]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

loader = CSVLoader(
    file_path=f"./data/Projects_with_Domains.csv",
    metadata_columns=[
      "Project Title",
      "Project Domain",
      "Secondary Domain",
      "Description",
      "Judge Comments",
      "Score",
      "Project Name",
      "Judge Score"
    ]
)

synthetic_usecase_data = loader.load()

for doc in synthetic_usecase_data:
    doc.page_content = doc.metadata["Description"]

Let's look at an example document to see if everything worked as expected!

In [16]:
synthetic_usecase_data[0]

Document(metadata={'source': './data/Projects_with_Domains.csv', 'row': 0, 'Project Title': 'InsightAI 1', 'Project Domain': 'Security', 'Secondary Domain': 'Finance / FinTech', 'Description': 'A low-latency inference system for multimodal agents in autonomous systems.', 'Judge Comments': 'Technically ambitious and well-executed.', 'Score': '85', 'Project Name': 'Project Aurora', 'Judge Score': '9.5'}, page_content='A low-latency inference system for multimodal agents in autonomous systems.')

## Task 3: Setting up QDrant!

Now that we have our documents, let's create a QDrant VectorStore with the collection name "Synthetic_Usecases".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

> NOTE: We'll be creating additional vectorstores where necessary, but this pattern is still extremely useful.

In [17]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Qdrant.from_documents(
    synthetic_usecase_data,
    embeddings,
    location=":memory:",
    collection_name="Synthetic_Usecases"
)

## Task 4: Naive RAG Chain

Since we're focusing on the "R" in RAG today - we'll create our Retriever first.

### R - Retrieval

This naive retriever will simply look at each review as a document, and use cosine-similarity to fetch the 10 most relevant documents.

> NOTE: We're choosing `10` as our `k` here to provide enough documents for our reranking process later

In [18]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

### A - Augmented

We're going to go with a standard prompt for our simple RAG chain today! Nothing fancy here, we want this to mostly be about the Retrieval process.

In [19]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### G - Generation

We're going to leverage `gpt-4.1-nano` as our LLM today, as - again - we want this to largely be about the Retrieval process.

In [20]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4.1-nano")

### LCEL RAG Chain

We're going to use LCEL to construct our chain.

> NOTE: This chain will be exactly the same across the various examples with the exception of our Retriever!

In [21]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's see how this simple chain does on a few different prompts.

> NOTE: You might think that we've cherry picked prompts that showcase the individual skill of each of the retrieval strategies - you'd be correct!

In [22]:
naive_retrieval_chain.invoke({"question" : "What is the most common project domain?"})["response"].content

'The most common project domain in the provided data appears to be "Healthcare / MedTech," which is mentioned multiple times.'

In [23]:
naive_retrieval_chain.invoke({"question" : "Were there any usecases about security?"})["response"].content

'Yes, there are usecases related to security. Specifically, one project titled "Pathfinder 24" is described as an "AI-powered platform optimizing logistics routes for sustainability" with the secondary domain of security.'

In [24]:
naive_retrieval_chain.invoke({"question" : "What did judges have to say about the fintech projects?"})["response"].content

'Judges had various comments about the fintech-related projects. For example, the project "TrendLens 19" was described as "Technically ambitious and well-executed," and "Pathfinder 27" received praise for "Excellent code quality and use of open-source libraries," with a high judge score of 9.8. Another project, "SecureNest 28," was noted as "Conceptually strong but results need more benchmarking," with a judge score of 9.0. Overall, judges recognized these fintech projects for their technical ambition, quality, and impact, though some noted areas for improvement in benchmarking and results.'

Overall, this is not bad! Let's see if we can make it better!

## Task 5: Best-Matching 25 (BM25) Retriever

Taking a step back in time - [BM25](https://www.nowpublishers.com/article/Details/INR-019) is based on [Bag-Of-Words](https://en.wikipedia.org/wiki/Bag-of-words_model) which is a sparse representation of text.

In essence, it's a way to compare how similar two pieces of text are based on the words they both contain.

This retriever is very straightforward to set-up! Let's see it happen down below!


In [25]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(synthetic_usecase_data)

We'll construct the same chain - only changing the retriever.

In [26]:
bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at the responses!

In [27]:
bm25_retrieval_chain.invoke({"question" : "What is the most common project domain?"})["response"].content

'Based on the provided data, the most common project domain appears to be "Finance / FinTech," as it is explicitly mentioned in multiple entries.'

In [28]:
bm25_retrieval_chain.invoke({"question" : "Were there any usecases about security?"})["response"].content

'Yes, there is a use case related to security. The project "SecureNest" in the domain of E‑commerce / Marketplaces and Secondary Domain of Legal / Compliance focuses on a document summarization and retrieval system for enterprise knowledge bases, which can be associated with security and compliance considerations.'

In [29]:
bm25_retrieval_chain.invoke({"question" : "What did judges have to say about the fintech projects?"})["response"].content

'The judges described the fintech projects positively. Specifically, for the project "SynthMind," which is in the finance/fintech domain, the judges said it had a "conceptually strong" approach, although it needed more benchmarking of results.'

It's not clear that this is better or worse, if only we had a way to test this (SPOILERS: We do, the second half of the notebook will cover this)

#### ❓ Question #1:

Give an example query where BM25 is better than embeddings and justify your answer.

##### ✅ Answer
BM25 is a traditional full-text search algorithm. It is useful when dealing with queries that rely on exact keyword matches. BM25 uses both term frequency - how often the term appears in the document and IDF - Inverse Document Frequency, how rare the term is across all documents, to identify documents.

It finds use in web searches, e-commerce websites and document retrieval.

An example suitable for BM25 is: "Find projects that use federated learning toolkit"

BM25 finds matches for the following reasons:
1. Exact terminilogy: BM25 finds exact matches for "federated", "learning", and "toolkit"
2. Rare terms "federated" appears in 4 documents, giving it massive BM25 weight
3. Technical precision: user wants this specific technique, not semantically similar ones
4. Embedding risk: might return generic "ML frameworks" or "privacy tools" instead



In [32]:
bm25_retrieval_chain.invoke({"question" : "Find projects that use federated learning toolkit"})["response"].content

'The projects that use a federated learning toolkit are:\n\n1. SkyForge\n2. SynthMind\n3. GreenPulse\n4. PixelSense\n\nThese projects are noted for improving privacy in healthcare applications using federated learning techniques.'

## Task 6: Contextual Compression (Using Reranking)

Contextual Compression is a fairly straightforward idea: We want to "compress" our retrieved context into just the most useful bits.

There are a few ways we can achieve this - but we're going to look at a specific example called reranking.

The basic idea here is this:

- We retrieve lots of documents that are very likely related to our query vector
- We "compress" those documents into a smaller set of *more* related documents using a reranking algorithm.

We'll be leveraging Cohere's Rerank model for our reranker today!

All we need to do is the following:

- Create a basic retriever
- Create a compressor (reranker, in this case)

That's it!

Let's see it in the code below!

In [33]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-v3.5")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

Let's create our chain again, and see how this does!

In [34]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [35]:
contextual_compression_retrieval_chain.invoke({"question" : "What is the most common project domain?"})["response"].content

'The most common project domain in the provided data appears to be "Synthetic_Usecases," as all the examples listed are related to synthetic data generators for various low-resource domain adaptation tasks. However, among the specific project domains mentioned in the sample, the ones listed are "Productivity Assistants," "Security," and "Healthcare / MedTech." \n\nSince the question asks about the most common project domain overall, and considering the context shows multiple entries under "Synthetic_Usecases" (which seems to be the collection or category), I would say that "Synthetic_Usecases" is the most common project domain in this dataset.\n\nIf you\'re referring specifically to the "Project Domain" field within the documents, the sample shows variability, and I do not have enough data to determine a single most common domain solely from the dataset. Based on the context, the overarching collection appears to focus on synthetic data use cases.\n\nTherefore, the most common project 

In [36]:
contextual_compression_retrieval_chain.invoke({"question" : "Were there any usecases about security?"})["response"].content

'Based on the provided context, there are no specific use cases related to security mentioned in the documents. The projects focus on areas like privacy in healthcare applications through federated learning, but there is no explicit mention of security-related use cases.'

In [37]:
contextual_compression_retrieval_chain.invoke({"question" : "What did judges have to say about the fintech projects?"})["response"].content

'The judges had a positive view of the fintech projects. Specifically, for the project "Pathfinder 27" in the Finance / FinTech domain, the judges commented on the "excellent code quality and use of open-source libraries," indicating a high level of technical proficiency. The project also received a high judge score of 9.8 out of 10.'

We'll need to rely on something like Ragas to help us get a better sense of how this is performing overall - but it "feels" better!

## Task 7: Multi-Query Retriever

Typically in RAG we have a single query - the one provided by the user.

What if we had....more than one query!

In essence, a Multi-Query Retriever works by:

1. Taking the original user query and creating `n` number of new user queries using an LLM.
2. Retrieving documents for each query.
3. Using all unique retrieved documents as context

So, how is it to set-up? Not bad! Let's see it down below!



In [38]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
) 

In [39]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [40]:
multi_query_retrieval_chain.invoke({"question" : "What is the most common project domain?"})["response"].content

'The most common project domain in the provided data appears to be "Healthcare / MedTech," which is mentioned multiple times among the projects.'

In [41]:
multi_query_retrieval_chain.invoke({"question" : "Were there any usecases about security?"})["response"].content

'Yes, there are use cases related to security. Specifically, one project titled "EchoLens" is described as a hardware-aware model quantization benchmark suite, which is related to security in the context of legal and compliance domains.'

In [42]:
multi_query_retrieval_chain.invoke({"question" : "What did judges have to say about the fintech projects?"})["response"].content

'The judges had mixed feedback on the fintech projects. For the project "Pathfinder 27," judges praised it for excellent code quality and use of open-source libraries, giving it a high score of 9.8. Overall, the comments suggest positive recognition for technical quality and innovation in the fintech projects.'

#### ❓ Question #2:

Explain how generating multiple reformulations of a user query can improve recall.

##### ✅ Answer
Each query reformulation captures different semantic nuances. So, together they cover more semantic space

1. Synonym coverage: Different words for the same concept
2. Semantic diversity: Multiple embedding vectors cover more space
3. Ambiguity resolution: Different interpretations of the query
4. Terminology variations: Abbreviations, full terms, alternatives
5. Perspective shifts: Different angles on the same question


## Task 8: Parent Document Retriever

A "small-to-big" strategy - the Parent Document Retriever works based on a simple strategy:

1. Each un-split "document" will be designated as a "parent document" (You could use larger chunks of document as well, but our data format allows us to consider the overall document as the parent chunk)
2. Store those "parent documents" in a memory store (not a VectorStore)
3. We will chunk each of those documents into smaller documents, and associate them with their respective parents, and store those in a VectorStore. We'll call those "child chunks".
4. When we query our Retriever, we will do a similarity search comparing our query vector to the "child chunks".
5. Instead of returning the "child chunks", we'll return their associated "parent chunks".

Okay, maybe that was a few steps - but the basic idea is this:

- Search for small documents
- Return big documents

The intuition is that we're likely to find the most relevant information by limiting the amount of semantic information that is encoded in each embedding vector - but we're likely to miss relevant surrounding context if we only use that information.

Let's start by creating our "parent documents" and defining a `RecursiveCharacterTextSplitter`.

In [43]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_docs = synthetic_usecase_data
child_splitter = RecursiveCharacterTextSplitter(chunk_size=750)

We'll need to set up a new QDrant vectorstore - and we'll use another useful pattern to do so!

> NOTE: We are manually defining our embedding dimension, you'll need to change this if you're using a different embedding model.

In [44]:
from langchain_qdrant import QdrantVectorStore

client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="full_documents",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = QdrantVectorStore(
    collection_name="full_documents", embedding=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

Now we can create our `InMemoryStore` that will hold our "parent documents" - and build our retriever!

In [45]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore = parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

By default, this is empty as we haven't added any documents - let's add some now!

In [46]:
parent_document_retriever.add_documents(parent_docs, ids=None)

We'll create the same chain we did before - but substitute our new `parent_document_retriever`.

In [47]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's give it a whirl!

In [48]:
parent_document_retrieval_chain.invoke({"question" : "What is the most common project domain?"})["response"].content

'Based on the provided data, the most common project domain appears to be "Healthcare / MedTech," as it is listed in at least one of the sample entries. However, since only a few entries are shown, I cannot definitively determine the most common domain overall. If you have access to the full dataset, you could analyze the frequency of each domain to find the most common one.'

In [49]:
parent_document_retrieval_chain.invoke({"question" : "Were there any usecases about security?"})["response"].content

'Based on the provided context, there are no specific use cases about security explicitly mentioned. The projects primarily focus on federated learning to improve privacy in healthcare applications, but explicit security use cases are not detailed.'

In [50]:
parent_document_retrieval_chain.invoke({"question" : "What did judges have to say about the fintech projects?"})["response"].content

'The judges\' comments about the fintech projects were generally positive. They described the projects as promising, clever, comprehensive, and technically mature. For example, one project was called "promising idea with robust experimental validation," another was noted for being "a clever solution with measurable environmental benefit," and others were described as "technically ambitious and well-executed" or "comprehensive and technically mature."'

Overall, the performance *seems* largely the same. We can leverage a tool like [Ragas]() to more effectively answer the question about the performance.

## Task 9: Ensemble Retriever

In brief, an Ensemble Retriever simply takes 2, or more, retrievers and combines their retrieved documents based on a rank-fusion algorithm.

In this case - we're using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

Setting it up is as easy as providing a list of our desired retrievers - and the weights for each retriever.

In [51]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

We'll pack *all* of these retrievers together in an ensemble.

In [52]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at our results!

In [53]:
ensemble_retrieval_chain.invoke({"question" : "What is the most common project domain?"})["response"].content

'The most common project domain in the provided data is "Finance / FinTech," which appears multiple times.'

In [54]:
ensemble_retrieval_chain.invoke({"question" : "Were there any usecases about security?"})["response"].content

'Yes, there are usecases related to security. For example, one project is "PixelSense," which is described as a "document summarization and retrieval system for enterprise knowledge bases," involving legal and compliance domains, which can be related to security and data privacy.'

In [55]:
ensemble_retrieval_chain.invoke({"question" : "What did judges have to say about the fintech projects?"})["response"].content

'The judges\' comments on the fintech projects were generally positive, highlighting their strength and potential. For example, one fintech project, SynthMind, was described as having a "conceptually strong" approach, though it needed more benchmarking results. Another fintech project, PulseAI, was noted for being "technically ambitious and well-executed." Overall, the judges recognized the innovative ideas and technical quality of the fintech projects.'

## Task 10: Semantic Chunking

While this is not a retrieval method - it *is* an effective way of increasing retrieval performance on corpora that have clean semantic breaks in them.

Essentially, Semantic Chunking is implemented by:

1. Embedding all sentences in the corpus.
2. Combining or splitting sequences of sentences based on their semantic similarity based on a number of [possible thresholding methods](https://python.langchain.com/docs/how_to/semantic-chunker/):
  - `percentile`
  - `standard_deviation`
  - `interquartile`
  - `gradient`
3. Each sequence of related sentences is kept as a document!

Let's see how to implement this!

We'll use the `percentile` thresholding method for this example which will:

Calculate all distances between sentences, and then break apart sequences of setences that exceed a given percentile among all distances.

In [56]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

Now we can split our documents.

In [57]:
semantic_documents = semantic_chunker.split_documents(synthetic_usecase_data[:20])

Let's create a new vector store.

In [58]:
semantic_vectorstore = Qdrant.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="Synthetic_Usecase_Data_Semantic_Chunks"
)

We'll use naive retrieval for this example.

In [59]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

Finally we can create our classic chain!

In [60]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

And view the results!

In [61]:
semantic_retrieval_chain.invoke({"question" : "What is the most common project domain?"})["response"].content

'Based on the provided data, the most common project domain appears to be "Legal / Compliance," as it is mentioned multiple times. However, given the small sample, I cannot definitively say it is the most frequent overall.'

In [62]:
semantic_retrieval_chain.invoke({"question" : "Were there any usecases about security?"})["response"].content

'Yes, there are use cases related to security. Specifically, one project named "BioForge" is a medical imaging solution in the security domain, and another project titled "InsightAI 1" called "Project Aurora" involves a low-latency inference system for multimodal agents in autonomous systems, which can be relevant to security applications.'

In [63]:
semantic_retrieval_chain.invoke({"question" : "What did judges have to say about the fintech projects?"})["response"].content

'Judges\' comments about the fintech projects generally highlighted their technical ambition, execution, and potential impact. Specifically:\n\n- The project "TrendLens 19" was described as "Technically ambitious and well-executed."\n- "WealthifyAI 16" was noted for its "Comprehensive and technically mature approach."\n- For "AutoMate 5," judges called it "A forward-looking idea with solid supporting data."\n\nAdditionally, some projects received praise for increased clarity and potential, such as "PlanPilot 7," which was described as having "Great clarity in communication and demo flow." Overall, judges recognized the innovative and well-implemented nature of these fintech projects, emphasizing their technical quality and promising outlook.'

#### ❓ Question #3:

If sentences are short and highly repetitive (e.g., FAQs), how might semantic chunking behave, and how would you adjust the algorithm?

##### ✅ Answer

1. percentile

How it works: Splits when distance exceeds the 95th percentile of all distances.

- Best for FAQs
- Only splits at major topic boundaries
- Ignores small variations in phrasing
- Groups similar Q&As together even if worded differently

Use when: Questions are repetitive but you want to group similar topics


2. standard_deviation

How it works: Splits when distance > mean + (multiplier × std_dev)
- Too sensitive
- Creates many small chunks
- Splits on minor phrasing differences

Use when: Content has consistent variation (not FAQs)

3. interquartile

How it works: Splits when distance > Q3 + (1.5 × IQR)
- Second best for FAQs
- Robust to outliers
- Groups similar questions
- Less aggressive than percentile

Use when: You want balanced chunking with outlier resistance

4. gradient

How it works: Splits when the change in distance suddenly increases
- Can be inconsistent
- Good at finding abrupt topic changes
- May split unnecessarily if FAQs aren't ordered by topic
- Works well if FAQs are grouped (all hours questions together)

Use when: Content has clear narrative progression or topic clustering


Best Choice: percentile (90-95th)

- Groups similar questions regardless of phrasing
- Only splits when topic truly changes
- Handles repetition gracefully




# 🤝 Breakout Room Part #2

#### 🏗️ Activity #1

Your task is to evaluate the various Retriever methods against eachother.

You are expected to:

1. Create a "golden dataset"
 - Use Synthetic Data Generation (powered by Ragas, or otherwise) to create this dataset
2. Evaluate each retriever with *retriever specific* Ragas metrics
 - Semantic Chunking is not considered a retriever method and will not be required for marks, but you may find it useful to do a "semantic chunking on" vs. "semantic chunking off" comparision between them
3. Compile these in a list and write a small paragraph about which is best for this particular data and why.

Your analysis should factor in:
  - Cost
  - Latency
  - Performance

> NOTE: This is **NOT** required to be completed in class. Please spend time in your breakout rooms creating a plan before moving on to writing code.

##### HINTS:

- LangSmith provides detailed information about latency and cost.

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import evaluate_retrievers

In [2]:
evaluate_retrievers.main()

2025-10-13 21:11:16.395 | INFO     | evaluate_retrievers:main:401 - Loading CSV dataset
2025-10-13 21:11:16.398 | INFO     | dataset_persistence:load:252 - Loading from valid cache: 54e0f4b7c97d12d0727bba81fe55268b.pkl
2025-10-13 21:11:16.402 | INFO     | evaluate_retrievers:main:404 - Generate test dataset
2025-10-13 21:11:17.087 | INFO     | testset_persistence:generate_with_cache:163 - Loading test dataset from cache (key: 8dcc5e6e...)
2025-10-13 21:11:17.089 | INFO     | testset_persistence:generate_with_cache:167 - Created: 2025-10-11T21:22:02.058591
2025-10-13 21:11:17.090 | INFO     | testset_persistence:generate_with_cache:168 - Size: 10 samples
2025-10-13 21:11:17.091 | INFO     | testset_persistence:generate_with_cache:169 - Docs: 50
2025-10-13 21:11:17.096 | INFO     | evaluate_retrievers:main:412 - Size of test dataset - 10
2025-10-13 21:11:17.097 | INFO     | evaluate_retrievers:main:415 - Create Qdrant vector store
2025-10-13 21:11:20.603 | INFO     | evaluate_retrievers:

Evaluating:   0%|          | 0/30 [00:00<?, ?it/s]

Exception raised in Job[2]: TimeoutError()
Exception raised in Job[5]: TimeoutError()
Exception raised in Job[17]: TimeoutError()
Exception raised in Job[20]: TimeoutError()
Exception raised in Job[23]: TimeoutError()
2025-10-13 21:18:42.651 | INFO     | evaluate_retrievers:main:507 - Results: {'context_recall': 1.0, 'context_entity_recall': 0.2941347537183415, 'noise_sensitivity_relevant': 0.28}
2025-10-13 21:18:42.652 | INFO     | evaluate_retrievers:main:508 - Latency: 422.58s
2025-10-13 21:18:42.653 | INFO     | evaluate_retrievers:main:454 - 
2025-10-13 21:18:42.654 | INFO     | evaluate_retrievers:main:455 - Evaluating: bm25_retriever
2025-10-13 21:18:42.655 | INFO     | evaluate_retrievers:main:456 - ============================================================
2025-10-13 21:18:42.657 | INFO     | evaluate_retrievers:main:465 - Expected project: Advanced-Retrieval-Eval-aae560d6-bm25_retriever
2025-10-13 21:18:42.659 | INFO     | evaluate_retrievers:main:470 - Generate evaluation 

Evaluating:   0%|          | 0/30 [00:00<?, ?it/s]

Exception raised in Job[2]: AttributeError('StringIO' object has no attribute 'statements')
Exception raised in Job[23]: AttributeError('StringIO' object has no attribute 'statements')
Exception raised in Job[17]: TimeoutError()
2025-10-13 21:25:27.060 | INFO     | evaluate_retrievers:main:507 - Results: {'context_recall': 0.85, 'context_entity_recall': 0.36464313417168415, 'noise_sensitivity_relevant': 0.08521303258145363}
2025-10-13 21:25:27.061 | INFO     | evaluate_retrievers:main:508 - Latency: 404.40s
2025-10-13 21:25:27.061 | INFO     | evaluate_retrievers:main:454 - 
2025-10-13 21:25:27.062 | INFO     | evaluate_retrievers:main:455 - Evaluating: contextual_compression_retriever
2025-10-13 21:25:27.063 | INFO     | evaluate_retrievers:main:456 - ============================================================
2025-10-13 21:25:27.065 | INFO     | evaluate_retrievers:main:465 - Expected project: Advanced-Retrieval-Eval-aae560d6-contextual_compression_retriever
2025-10-13 21:25:27.066 

...

Exception raised in Job[29]: AttributeError('StringIO' object has no attribute 'statements')
Exception raised in Job[2]: TimeoutError()
Exception raised in Job[5]: TimeoutError()
Exception raised in Job[17]: TimeoutError()
Exception raised in Job[20]: TimeoutError()
Exception raised in Job[23]: TimeoutError()
2025-10-13 21:40:14.758 | INFO     | evaluate_retrievers:main:507 - Results: {'context_recall': 1.0, 'context_entity_recall': 0.31809829015486324, 'noise_sensitivity_relevant': 0.2669172932330827}
2025-10-13 21:40:14.758 | INFO     | evaluate_retrievers:main:508 - Latency: 476.45s
2025-10-13 21:40:14.759 | INFO     | evaluate_retrievers:main:454 - 
2025-10-13 21:40:14.760 | INFO     | evaluate_retrievers:main:455 - Evaluating: parent_document_retriever
2025-10-13 21:40:14.761 | INFO     | evaluate_retrievers:main:456 - ============================================================
2025-10-13 21:40:14.762 | INFO     | evaluate_retrievers:main:465 - Expected project: Advanced-Retrieva

Evaluating:   0%|          | 0/30 [00:00<?, ?it/s]

2025-10-13 21:47:12.970 | INFO     | evaluate_retrievers:main:507 - Results: {'context_recall': 1.0, 'context_entity_recall': 0.30216394678622677, 'noise_sensitivity_relevant': 0.10478773273083215}
2025-10-13 21:47:12.972 | INFO     | evaluate_retrievers:main:508 - Latency: 418.20s
2025-10-13 21:47:12.972 | INFO     | evaluate_retrievers:main:454 - 
2025-10-13 21:47:12.974 | INFO     | evaluate_retrievers:main:455 - Evaluating: ensemble_retriever
2025-10-13 21:47:12.974 | INFO     | evaluate_retrievers:main:456 - ============================================================
2025-10-13 21:47:12.977 | INFO     | evaluate_retrievers:main:465 - Expected project: Advanced-Retrieval-Eval-aae560d6-ensemble_retriever
2025-10-13 21:47:12.979 | INFO     | evaluate_retrievers:main:470 - Generate evaluation dataset for `ensemble_retriever`
2025-10-13 21:47:12.982 | INFO     | evaluate_retrievers:create_evaluation_dataset:260 - Make LCEL RAG chain for `ensemble_retriever`
2025-10-13 21:49:58.941 | I

Evaluating:   0%|          | 0/30 [00:00<?, ?it/s]

Exception raised in Job[2]: TimeoutError()
Exception raised in Job[5]: TimeoutError()
Exception raised in Job[8]: TimeoutError()
Exception raised in Job[17]: TimeoutError()
Exception raised in Job[20]: TimeoutError()
Exception raised in Job[23]: TimeoutError()
Exception raised in Job[29]: TimeoutError()
2025-10-13 21:56:12.754 | INFO     | evaluate_retrievers:main:507 - Results: {'context_recall': 1.0, 'context_entity_recall': 0.3172649568272938, 'noise_sensitivity_relevant': 0.37896825396825395}
2025-10-13 21:56:12.755 | INFO     | evaluate_retrievers:main:508 - Latency: 539.77s
2025-10-13 21:56:12.756 | INFO     | evaluate_retrievers:main:514 - 
2025-10-13 21:56:12.757 | INFO     | evaluate_retrievers:main:515 - FETCHING COST DATA FROM LANGSMITH
2025-10-13 21:56:12.757 | INFO     | evaluate_retrievers:main:516 - ============================================================
2025-10-13 21:56:12.759 | INFO     | evaluate_retrievers:main:531 - Session ID: aae560d6
2025-10-13 21:56:20.987 

In [3]:
### YOUR CODE HERE
# Check evaluate_retrievers.py
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv("data/retriever_evaluation_results.csv")

In [5]:
df['avg_performance'] = (df["context_recall"] + df["context_entity_recall"] + df["noise_sensitivity"])/3
df['norm_avg_performance'] = (df['avg_performance'] - df['avg_performance'].min()) / (df['avg_performance'].max() - df['avg_performance'].min())
df['norm_latency_seconds'] = (df['langsmith_total_latency'] - df['langsmith_total_latency'].min()) / (df['langsmith_total_latency'].max() - df['langsmith_total_latency'].min())
df['norm_cost_usd'] = (df['langsmith_total_cost_usd'] - df['langsmith_total_cost_usd'].min()) / (df['langsmith_total_cost_usd'].max() - df['langsmith_total_cost_usd'].min())

### Comprehensive Comparison Table

In [6]:
# Highlight max/min only
simple_highlight = df.style.highlight_max(
    subset=['context_recall', 'context_entity_recall', 'noise_sensitivity', 'avg_performance'],
    color='lightgreen'
).highlight_min(
    subset=['langsmith_total_latency', 'langsmith_total_cost_usd'],
    color='lightgreen'
).highlight_min(
    subset=['context_recall', 'context_entity_recall', 'noise_sensitivity', 'avg_performance'],
    color='lightcoral'
).highlight_max(
    subset=['langsmith_total_latency', 'langsmith_total_cost_usd'],
    color='lightcoral'
)

simple_highlight


,retriever,context_recall,context_entity_recall,noise_sensitivity,latency_seconds,langsmith_total_cost_usd,langsmith_total_tokens,langsmith_llm_calls,langsmith_total_latency,avg_performance,norm_avg_performance,norm_latency_seconds,norm_cost_usd
0,naive_retriever,1.000000,0.294135,0.280000,422.575179,0.354360,458689,11,419.819300,0.524712,0.691964,0.131548,0.703636
1,bm25_retriever,0.850000,0.364643,0.085213,404.397298,0.204297,256670,11,402.144392,0.433285,0.000000,0.000000,0.034727
2,contextual_compression_retriever,1.000000,0.459648,0.209494,411.234186,0.196506,241127,11,408.147456,0.556381,0.931654,0.044679,0.000000
3,multi_query_retriever,1.000000,0.318098,0.266917,476.448286,0.389584,506027,11,473.720443,0.528339,0.719415,0.532713,0.860646
4,parent_document_retriever,1.000000,0.302164,0.104788,418.203961,0.235850,285475,11,413.325875,0.468984,0.270186,0.083219,0.175375
5,ensemble_retriever,1.000000,0.317265,0.378968,539.772266,0.420847,557117,11,536.505738,0.565411,1.000000,1.000000,1.000000



WINNER: Contextual Compression offers the best cost-performance-latency balance, performing within 2% of the best (Ensemble) while being significantly cheaper and faster - within 2% of the fastest (BM25) retriever.

Why:<br>
**Performance Leader:** Ensemble (0.565)

- Highest average performance
- But: Expensive and slow

**Speed Leader:** BM25 (402s)

- Fastest retrieval
- But: Lowest performance

**Cost Leader:** Contextual Compression ($0.19)

- Cheapest option
- Decent performance (0.556) - only 1.6% worse than Ensemble (best performer)
- Good latency (408s) - only 1.5% slower than BM25 (fastest)

**Actual Winner:** It Depends on Priority

If Performance is Critical → Ensemble

- Accept higher cost/latency for best results

If Speed is Critical → BM25

- Accept poor performance for fastest retrieval

If Balanced/Cost-Conscious → Contextual Compression

- Best cost-performance-latency tradeoff
- Only marginally worse than Ensemble (0.556 vs 0.565)
- Much cheaper and faster, only slight slower than BM25 (402s vs 408s)

We could consider improving the results by adding lengthier descriptions as the original descriptions were less than 100 tokens. We could also consider increasing the testset sample size which could help improve performance albeit increasing both latency and cost